<a href="https://colab.research.google.com/github/babazeedy/math-for-machine-learning/blob/main/Copy_of_MScFE_610_GWP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Answer 3a.**

We will use two distinct approaches:
* **Backward Elimination** using the **Bayesian Information Criterion (BIC)**.<br>
* **All Subsets Regression** using the **Adjusted $R^2$** and the **Akaike Information Criterion (AIC).**

This is to balance goodness-of-fit with model complexity.

1. Data Loading and Preparation

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import itertools

In [ ]:
df = pd.read_csv("FE-GWP1_model_selection_2.csv")

In [ ]:
df.columns = ['Y', 'Z1', 'Z2', 'Z3', 'Z4', 'Z5']

In [ ]:
# Define Y and X
Y = df['Y']
X = df[['Z1', 'Z2', 'Z3', 'Z4', 'Z5']]

In [ ]:
# Add constant for the intercept term (required by statsmodels)
X = sm.add_constant(X, prepend=False)

In [ ]:
# Available predictors (excluding the constant)
predictors = ['Z1', 'Z2', 'Z3', 'Z4', 'Z5']

In [ ]:
print(f"Dataset loaded. Sample size (N): {len(df)}")

Dataset loaded. Sample size (N): 100


* **First Approach**<br>
**Backward Elimination (Using BIC)**<br>
**Justification**: Backward elimination systematically removes the least useful predictor at each step. By using the **Bayesian Information Criterion (BIC)**, we prioritize **parsimony** (simplicity). BIC heavily penalizes models with more parameters ($k$) than AIC, favoring the simplest model that adequately explains the data. We aim to **minimize BIC.**

In [ ]:
def backward_elimination_bic(X, Y, initial_vars):
    """Performs backward elimination using BIC as the criterion."""

    selected_vars = initial_vars + ['const']

    # 1. Start with the full model
    model_full = sm.OLS(Y, X[selected_vars]).fit()
    best_bic = model_full.bic
    best_model_vars = selected_vars.copy()

    print("\n--- First approach: Backward Elimination (Optimizing BIC) ---")
    print(f"Starting BIC (Full Model: {len(initial_vars)} vars): {best_bic:.3f}")

    while len(selected_vars) > 1: # Stop when only intercept remains

        candidates_to_remove = [var for var in selected_vars if var != 'const']
        bic_scores = {}

        # Check BIC for models formed by removing one variable
        for var_to_remove in candidates_to_remove:
            current_vars = [v for v in selected_vars if v != var_to_remove]
            model = sm.OLS(Y, X[current_vars]).fit()
            bic_scores[var_to_remove] = model.bic

        # Find the variable whose removal results in the lowest BIC
        best_removal = min(bic_scores, key=bic_scores.get)
        new_bic = bic_scores[best_removal]

        print(f"\nEvaluating removals from {len(selected_vars)-1} vars: {candidates_to_remove}")

        # Check stopping condition: only proceed if BIC improves (decreases)
        if new_bic < best_bic:
            print(f"Removed {best_removal} -> New BIC: {new_bic:.3f} (IMPROVEMENT)")
            selected_vars.remove(best_removal)
            best_bic = new_bic
            best_model_vars = selected_vars.copy()
        else:
            print(f"Removing {best_removal} -> New BIC: {new_bic:.3f} (WORSENING) -> STOP.")
            break

    final_vars = [v for v in best_model_vars if v != 'const']
    return final_vars, best_bic

# Execute Backward Elimination
bic_model_vars, best_bic_score = backward_elimination_bic(X, Y, predictors)


--- First approach: Backward Elimination (Optimizing BIC) ---
Starting BIC (Full Model: 5 vars): -150.271

Evaluating removals from 5 vars: ['Z1', 'Z2', 'Z3', 'Z4', 'Z5']
Removing Z1 -> New BIC: -105.160 (WORSENING) -> STOP.


**Conclusion from backward Elimination:**

The process correctly stopped immediately at Step 1:
* The BIC for the full model, $\mathbf{-150.271}$, is the lowest BIC found.
* Removing any single predictor causes the BIC to increase (i.e., become less negative), which signals a worsening of the model's overall fit/parsimony trade-off.

Therefore, the best model found by the Backward Elimination process is the Full Model $\{Z_1, Z_2, Z_3, Z_4, Z_5\}$, with a BIC score of $\mathbf{-150.271}$.

**Second approach:** <br>**All Subsets Regression (Using Adjusted $R^2$ and AIC)**<br>**Justification:** All Subsets Regression evaluates every possible combination of predictors, ensuring the true globally optimal subset for any given size is found. We will use two criteria for this approach:
* **Adjusted $R^2$ ($\bar{R}^2$)**: We aim to **maximize** $\bar{R}^2$, as it indicates the highest proportion of variance explained after accounting for the number of variables.
* **Akaike Information Criterion (AIC)**: We aim to **minimize AIC**, which provides a slightly less punitive penalty for complexity than BIC.

In [ ]:
def all_subsets_regression(X, Y, predictors):
    """Performs all subsets regression and selects the best models."""

    best_results = {'max_adj_r2': (-np.inf, []), 'min_aic': (np.inf, [])}

    print("\n---  Second approach: All Subsets Regression ---")

    # Iterate through all possible model sizes (k=1 to k=5)
    for k in range(1, len(predictors) + 1):

        # Generate all combinations of k predictors
        for subset in itertools.combinations(predictors, k):

            current_vars = list(subset) + ['const']
            X_current = X[current_vars]

            # Fit the model
            model = sm.OLS(Y, X_current).fit()

            # Calculate criteria
            adj_r2 = model.rsquared_adj
            aic = model.aic

            # Check for best Adjusted R^2
            if adj_r2 > best_results['max_adj_r2'][0]:
                best_results['max_adj_r2'] = (adj_r2, subset)

            # Check for best AIC
            if aic < best_results['min_aic'][0]:
                best_results['min_aic'] = (aic, subset)

    # Output results
    print(f"Total Models Evaluated: {2**len(predictors) - 1}")
    print("\nModel Selection Summary:")
    print(f"1. Best Model by MAX Adjusted R-Squared ({best_results['max_adj_r2'][0]:.4f}): {list(best_results['max_adj_r2'][1])}")
    print(f"2. Best Model by MIN AIC ({best_results['min_aic'][0]:.3f}): {list(best_results['min_aic'][1])}")

    return best_results['max_adj_r2'][1], best_results['min_aic'][1]

# Execute All Subsets Regression
adj_r2_model_vars, aic_model_vars = all_subsets_regression(X, Y, predictors)


---  Second approach: All Subsets Regression ---
Total Models Evaluated: 31

Model Selection Summary:
1. Best Model by MAX Adjusted R-Squared (0.9936): ['Z1', 'Z2', 'Z3', 'Z4', 'Z5']
2. Best Model by MIN AIC (-165.902): ['Z1', 'Z2', 'Z3', 'Z4', 'Z5']


In [ ]:
final_vars = ['Z1', 'Z2', 'Z3', 'Z4', 'Z5']
X_final = X[final_vars + ['const']]
final_model = sm.OLS(Y, X_final).fit()

print("\n--- Final Model (Z1, Z2, Z3, Z4, Z5) Summary ---")
print(final_model.summary())


--- Final Model (Z1, Z2, Z3, Z4, Z5) Summary ---
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     3062.
Date:                Mon, 15 Dec 2025   Prob (F-statistic):          2.07e-102
Time:                        12:31:59   Log-Likelihood:                 88.951
No. Observations:                 100   AIC:                            -165.9
Df Residuals:                      94   BIC:                            -150.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Z1

**Conclusion from the Results**<br>The OLS regression results show that the model:<br>
* **Is Highly Significant and Excellent Fit:** The model is collectively highly significant (F-stat) and explains virtually all of the variation in $Y$ ($\text{Adj.} R^2 = 0.994$).
* Every Predictor is Relevant: Every single variable ($Z_1$ through $Z_5$) has a highly statistically significant unique contribution to predicting $Y$ ($p < 0.001$).
* **Assumptions are Met:** Key assumptions regarding normality of residuals (Omnibus/JB tests) and autocorrelation (Durbin-Watson) are satisfied, and multicollinearity is not an issue.


The detailed summary confirms that the **Full Model** ($\{Z_1, Z_2, Z_3, Z_4, Z_5\}$) is the best and most appropriate model for this dataset.